# 뉴스 크롤링코드

## 사이트
- (naver)SBS
- Daily Gaewon
- (naver)KBS
- (naver)Kukmin Ilbo
- 키워드:반려동물복지 / 페이지갯수 :11개

## 실행환경
window에서 작업 -> VM ubunto 20에서 zeppelin으로 작업, pyspark+pands(특정 df병합에서사용)

## 변수명
워드카운트를 사용 /사용하지않은것으로 나뉘기때문에 wordx가붙은것이 서비스에이용 된 최종파일

In [ ]:
#Daily Gaewon

from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import FirefoxOptions
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pyspark.sql import *
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import col
import re


webdriver_service = Service('/usr/bin/geckodriver') #(윈도우)경로를 \ 로 복사하지말고, /로변경해줄것.

opts = FirefoxOptions()
opts.add_argument("--headless")
# opts.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'#(윈도우)에러,moz:firefoxOptions.binary를 해결하기위해, 파이어폭스 경로를 수동지정.
browser = webdriver.Firefox(service=webdriver_service, options=opts)

# 한 페이지내에서 모든 뉴스의 갯수만큼 타이틀 / 링크를 따로추출하여 두개의 리스트에 저장.
dailygaewon_list=[] #뉴스정보를 저장할 리스트
dailygawon_link=[] #본문링크만 저장할 리스트(본문크롤링에 필요하기에 따로 저장)

#페이지를 순환할 for문
for pages in range(1,7): #6페이지까지니까 +1해서 7
    base_url = f"http://www.dailygaewon.com/news/articleList.html?page={pages}&total=111&sc_section_code=&sc_sub_section_code=S2N30&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&box_idxno=&sc_multi_code=&sc_is_image=&sc_is_movie=&sc_order_by=E"
    browser.get(base_url)

    #페이지 내의 모든 기사의 갯수를추출하여 변수로 저장
    all_news=browser.find_elements(By.CLASS_NAME,"table-row")
    all_news_num=len(all_news)
    
    #기사의 길이만큼 for문을 반복해서 타이틀, 링크를 따로 추출
    for all_news_n in range(1,all_news_num+1): #1~19까지만 작동되니까 +1을해서 갯수를맞춰줌
        news_title=browser.find_element(By.XPATH,f"//*[@id='user-container']/div[3]/div[2]/section/article/div[2]/section/div[{all_news_n}]/div[1]/a/strong").text
        news_link=browser.find_element(By.XPATH,f"//*[@id='user-container']/div[3]/div[2]/section/article/div[2]/section/div[{all_news_n}]/div[1]/a").get_attribute("href")
        dailygaewon_list.append([news_title,news_link])
        dailygawon_link.append(news_link) #이것은 데이터 프레임으로 만들지 x
# print(dailygaewon_list)
# print(news_link)


#링크도 데이터프레임에 추가했음.
dailygawon_contents=[]
for dailygawon_links in dailygawon_link:
    browser.get(dailygawon_links)
    news_cont_title=browser.find_element(By.XPATH,f"//*[@id='user-container']/div[3]/header/div/div").text
    news_cont=browser.find_element(By.CSS_SELECTOR,"div#article-view-content-div").text
    news_date=browser.find_element(By.XPATH,f"//*[@id='user-container']/div[3]/header/section/div/ul/li[2]").text
    dailygawon_contents.append([news_cont_title,news_cont,news_date,dailygawon_links])
    # print(dailygawon_contents)
    
# print(dailygaewon_list)
print(dailygawon_contents)
browser.quit()   

In [ ]:
#크롤링데이터 DF로생성하고 내용 특정문자 삭제
dailygaewon_index = ["title","main","date","url"]
dailygaewon_data= spark.createDataFrame(dailygawon_contents,dailygaewon_index)
dailygaewon_data.createOrReplaceTempView("dailygaewon")
dailygaewon_data.show()

#main컬럼
main_p=dailygaewon_data.select(col("title"),regexp_replace(col("main"),'[■\n※-▶◆▼●▲『』]'," ").alias("main"),regexp_replace(col("date"),'승인',"").alias("date"),col("url"))

#title컬럼
title_p=main_p.select(regexp_replace(col("title"),'\[[^)]*\]',"").alias("title"),col("main"),regexp_replace(col("date"),'\[[^)]*\]',"").alias("date"),col("url"))
title_p.show()

In [ ]:
#특정문자를 삭제하고 DF에 id를 부여
dailygaewon_wordcount_x=title_p.rdd.zipWithIndex().toDF()
final_dailygaewon_df_wordx=dailygaewon_wordcount_x.select(col("_2").alias("id"),col("_1.*")) #_2가 id. id가앞에오게 _2를 먼저부름
final_dailygaewon_df_wordx.show()

#하둡에저장할때 파티션을 나누지않고 병합
final_dailygaewon_df_wordx.coalesce(1).write.options(header='True', delimiter=',', encoding="cp949").csv("/dailygaewon/dailygaewon_final_url.csv/")


In [ ]:
#Kukmin Ilbo

base_url="http://www.dailygaewon.com/news/articleList.html?sc_sub_section_code=S2N30&view_type=sm"
webdriver_service = Service('/usr/bin/geckodriver')

opts = FirefoxOptions()
opts.add_argument("--headless")		
browser = webdriver.Firefox(service=webdriver_service, options=opts)


#네이버 페이지 크롤링용 변수

pagenum=[1,11,21,31,41,51,61,71,81,91,101]

#1.국민일보 크롤링 / 링크

Kukmin_link=[] #국민일보 본문링크만 담을 리스트

for links in pagenum:
    Kukmin_pagelink=f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EB%B0%98%EB%A0%A4%EB%8F%99%EB%AC%BC%20%EB%B3%B5%EC%A7%80&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=20&mynews=1&office_type=1&office_section_code=1&news_office_checked=1005&nso=so:r,p:all,a:all&start={links}"
    browser.get(Kukmin_pagelink)
    all_page_lang=browser.find_elements(By.CLASS_NAME,"dsc_thumb")
    for cont in all_page_lang:
        Kukmin_content_link=cont.get_attribute("href") #본문페이지 링크 추출
        Kukmin_link.append(Kukmin_content_link)
# print(Kukmin_link)
# print(len(Kukmin_link))

#======1-2. 국민일보 크롤링=============

#타이틀/ 본문 / 입력날짜 /url

Kukmin_main_text=[]


for Kukmin_links in Kukmin_link:
    browser.get(Kukmin_links) #본문페이지로 변경
    try:
        Kukmin_title=browser.find_element(By.CSS_SELECTOR, "div.nwsti h3").text
        Kukmin_main=browser.find_element(By.CLASS_NAME, "tx").text
        Kukmin_date=browser.find_element(By.CLASS_NAME, "t11").text
        Kukmin_main_text.append([Kukmin_title,Kukmin_main,Kukmin_date,Kukmin_links])
    except:
        pass #daily gaewon은 에러가나지않았지만, kbs,sbs,kukmin은 오류가있음
print(Kukmin_main_text)

browser.quit()

In [ ]:
#이후 전처리과정은 Daily gaewon과 비슷한 흐름

Kukmin_index = ["title", "main", "date","url"]
Kukmin_news = spark.createDataFrame(Kukmin_main_text, Kukmin_index)
Kukmin_news.createOrReplac

#main컬럼 전처리
main_p=Kukmin_news.select(col("title"),regexp_replace(col("main"),'[\n▶▲■▷▼●]'," ").alias("main"),col("date"),col("url"))
# title 컬럼
title_p=main_p.select(regexp_replace(col("title"),'[,]',"").alias("title"),col("main"),col("date"),col("url"))
title_p.show()

#id생성문
kukmin_wordcount_x=title_p.rdd.zipWithIndex().toDF()
final_kukmin_df_wordx=kukmin_wordcount_x.select(col("_2").alias("id"),col("_1.*"))
final_kukmin_df_wordx.show()
final_kukmin_df_wordx.coalesce(1).write.options(header='True', delimiter=',', encoding="cp949").csv("/naver/Kukmin_news_url2.csv/")

In [ ]:
# KBS

webdriver_service = Service('/usr/bin/geckodriver')

opts = FirefoxOptions()
opts.add_argument("--headless")		
browser = webdriver.Firefox(service=webdriver_service, options=opts)


#네이버 페이지 크롤링용 변수

pagenum=[1,11,21,31,41,51,61,71,81,91,101]

#1.KBS 크롤링 / 링크

kbs_link=[] # KBS 본문링크만 담을 리스트

for links in pagenum:
    kbs_pagelink=f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EB%B0%98%EB%A0%A4%EB%8F%99%EB%AC%BC%20%EB%B3%B5%EC%A7%80&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=28&mynews=1&office_type=1&office_section_code=2&news_office_checked=1056&nso=so:r,p:all,a:all&start={links}"
    browser.get(kbs_pagelink)
    all_page_lang=browser.find_elements(By.CLASS_NAME,"dsc_thumb")
    for cont in all_page_lang:
        kbs_content_link=cont.get_attribute("href")
        kbs_link.append(kbs_content_link)

#======1-2. KBS / 본문내용=============

#타이틀/ 본문 / 입력날짜 /URL
Kbs_main_text=[]
for kbs_links in kbs_link:
    try:
        browser.get(kbs_links) #본문페이지로 변경
        kbs_title=browser.find_element(By.CLASS_NAME, "tit-s").text
        kbs_main=browser.find_element(By.ID, "cont_newstext").text
        kbs_date=browser.find_element(By.CLASS_NAME, "date").text
        Kbs_main_text.append([kbs_title,kbs_main,kbs_date,kbs_links])
    except:
        pass
print(Kbs_main_text)
browser.quit()


In [ ]:
Kbs_index = ["title", "main", "date","url"]
Kbs_news = spark.createDataFrame(Kbs_main_text, Kbs_index)
Kbs_news.createOrReplaceTempView("Kbs_news")
Kbs_news.show()

#main컬럼 전처리
main_p=Kbs_news.select(col("title"),regexp_replace(col("main"),'[■\n※-▶◆▼●▲]'," ").alias("main"),regexp_replace(col("date"),'입력',"").alias("date"),col("url"))

#title컬럼 전처리
title_p=main_p.select(regexp_replace(col("title"),'[,]',"").alias("title"),col("main"),regexp_replace(col("date"),'\([^)]*\)',"").alias("date"),col("url"))
title_p.show()

#ID생성,CSV파일생성
kbs_wordcount_x=title_p.rdd.zipWithIndex().toDF()
final_kbs_df_wordx=kbs_wordcount_x.select(col("_2").alias("id"),col("_1.*"))
final_kbs_df_wordx.show()
final_kbs_df_wordx.coalesce(1).write.options(header='True', delimiter=',', encoding="cp949").csv("/naver/Kbs_news_url.csv/")

In [ ]:
#SBS

webdriver_service = Service('/usr/bin/geckodriver')

opts = FirefoxOptions()
opts.add_argument("--headless")		
browser = webdriver.Firefox(service=webdriver_service, options=opts)

#네이버 페이지 크롤링용 변수
pagenum=[1,11,21,31,41,51,61,71,81,91,101]

#1.SBS 크롤링 / 링크

sbs_link=[] 
for links in pagenum:
    sbs_pagelink=f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EB%B0%98%EB%A0%A4%EB%8F%99%EB%AC%BC%20%EB%B3%B5%EC%A7%80&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=22&mynews=1&office_type=1&office_section_code=2&news_office_checked=1055&nso=so:r,p:all,a:all&start={links}"
    browser.get(sbs_pagelink)
    all_page_lang=browser.find_elements(By.CLASS_NAME,"dsc_thumb")
    for cont in all_page_lang:
        sbs_content_link=cont.get_attribute("href") 
        sbs_link.append(sbs_content_link)

#======1-2. SBS 크롤링 / 본문내용=============

#타이틀/ 본문 / 입력날짜 /URL
sbs_main_text=[]
for sbs_links in sbs_link:
    browser.get(sbs_links) 
    sbs_title=browser.find_element(By.ID, "vmNewsTitle").text
    sbs_main=browser.find_element(By.CLASS_NAME, "text_area").text
    sbs_date=browser.find_element(By.XPATH, "//*[@id='container']/div[1]/div[2]/div[1]/div[1]/span/span[1]").text
    sbs_main_text.append([sbs_title,sbs_main,sbs_date,sbs_links])
    print(sbs_main_text)

browser.quit()


In [ ]:
sbs_index = ["title", "main", "date","url"]
sbs_news = spark.createDataFrame(sbs_main_text,sbs_index)
sbs_news.createOrReplaceTempView("sbs_news")
sbs_news.show()

#main컬럼 전처리
main_p=sbs_news.select(col("title"),regexp_replace(col("main"),'[\n▶▲■▷▼●\[\]]'," ").alias("main"),col("date"),col("url"))
# title 컬럼에대하여 정규식을 사용해 여러문자 제거. []는 특수한문자이므로 이스케이프가 필요하여 앞에 \를 사용. []안에있는 문자들과 일치하면 모두 공백으로변경
title_p=main_p.select(regexp_replace(col("title"),'[\[,"\]+]',"").alias("title"),col("main"),col("date"),col("url"))
title_p.show()

#id생성,csv파일 생성

sbs_rdd_id=title_p.rdd.zipWithIndex().toDF()
final_sbs_wordx=sbs_rdd_id.select(col("_2").alias("id"),col("_1.*"))
final_sbs_wordx.show()
final_sbs_wordx.coalesce(1).write.options(header='True', delimiter=',', encoding="cp949").csv('/naver/sbs_news_url.csv/')

## KBS,SBS,KUKMIN ILBO, DAYILY GAEWON.CSV 병합
Pandas사용

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import col, lit
import pandas as pd

#sbs/kbs/kukmin/dailygaewon csv load
kbs_news_read=spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("encoding","cp949").load('/naver/Kbs_news_url.csv/*.csv')
sbs_news_read=spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("encoding","cp949").load('/naver/sbs_news_url.csv/*.csv')
kukmin_news_read=spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("encoding","cp949").load('/naver/Kukmin_news_url.csv/*.csv')
dailygaewon_news_read=spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("encoding","cp949").load('/dailygaewon/dailygaewon_final_url.csv/*.csv')

#csv파일load확인작업
kbs_news_read.show()
sbs_news_read.show()
kukmin_news_read.show()
dailygaewon_news_read.show()

#각 언론사별로 태그가있으면 좋겠다는 요청이들어와, csv파일에 언론사명 추가작업
kbs_news_press=kbs_news_read.withColumn("press", lit("KBS"))
sbs_news_press=sbs_news_read.withColumn("press", lit("SBS"))
kukmin_news_press=kukmin_news_read.withColumn("press", lit("KukminIlbo"))
dailygaewon_news_press=dailygaewon_news_read.withColumn("press", lit("DailyGaewon"))

In [ ]:
#데이터프레임 병합작업
# kbs_idx.count() 104 
# sbs_idx.count() 95 
# kukmin_idx.count() 92 
# daily_idx.count() 111

#id제거
kbs_idx=kbs_news_press.select(col("title"),col("main"),col("date"),col("url"),col("press"))
sbs_idx=sbs_news_press.select(col("title"),col("main"),col("date"),col("url"),col("press"))
kukmin_idx=kukmin_news_press.select(col("title"),col("main"),col("date"),col("url"),col("press"))
daily_idx=dailygaewon_news_press.select(col("title"),col("main"),col("date"),col("url"),col("press"))

#concat을위해 sparkDF->pandasDF->sparkDF으로 변환 작업

result_kbs = kbs_idx.select("*").toPandas()
result_sbs = sbs_idx.select("*").toPandas()
result_kukmin = kukmin_idx.select("*").toPandas()
result_daily = daily_idx.select("*").toPandas()

concatDF=pd.concat([result_kbs,result_sbs,result_kukmin,result_daily])
news_concatdf = spark.createDataFrame(concatDF)

#ID재부여

news_concat_id=news_concatdf.rdd.zipWithIndex().toDF()
final_newsDF=news_concat_id.select(col("_2").alias("id"),col("_1.*"))

final_newsDF.show(402)

#save
final_newsDF.coalesce(1).write.options(header='True', delimiter=',', encoding="cp949").csv('/news/news_final.csv/')


## 뉴스산출물
- Kbs_news_url.csv
- sbs_news_url.csv
- kukminlibo_news_url.csv
- dailygaewon_final_url.csv
- news_final.csv(사용테이블1)

# 동물병원CSV파일 가공과정

In [ ]:
#동물병원 CSV 원본파일 PYSPARK를 통한 전처리작업
from pyspark.sql import *
from pyspark.sql.functions import col, lit
import pyspark.sql.functions 

#하둡에 저장할때 동물병원.CSV 의 이름을 medicaladdress.csv로 바꿔서저장하고 pyspark로 load
medical_read=spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("encoding","cp949").load('/medical.csv/medicaladdress.csv')

sort1=medical_read.select(col("사업장명"),col("소재지전체주소"),col("소재지전화"),col("좌표정보(x)"),col("좌표정보(y)"))
#sort1 /9589
sort1.show()

#비어있는 값이 있으면 무조건 해당 로우삭제.
# drop_null /6402
drop_null=sort1.na.drop("any")
drop_null.show(6403)

#약국과 병원 분리작업
#해당 단어포함시 총갯수 기록:병원만=5665/메디컬 포함=5845 /센터포함=6194

medical_filter=drop_null.filter(col('사업장명').like('%병원%') | col('사업장명').like('%메디컬%') | col('사업장명').like('%센터%'))
medical_filter.show(6197)

#medical_final_1.csv생성
medical_filter.coalesce(1).write.options(header='True', delimiter=',', encoding="cp949").csv('/medical/medical_final_1.csv/')



# Mysql 워크벤치로 medical_final_1.csv를 medical 테이블로 import

In [ ]:
# Pyspark로 전처리할때 빠트린 중복값 검색,제거, 중복제거로일어난 id값 재정렬 

#id제외 위도,경도 중복값 조회
SELECT lot,lat, COUNT(*) as cnt
FROM medical #위에서 만든 medical이라는 테이블
GROUP BY lot,lat
HAVING COUNT(lot) > 1 AND COUNT(lat) > 1;

#위의 셀렉트결과를 토대로 중복값 제거
DELETE t1 FROM medical t1 
JOIN medical t2
ON t1.lot=t2.lot AND t1.lat=t2.lat
WHERE t1.id > t2.id;

#삭제로 일정하지않은 id를 재생성
SET @CON =0;
UPDATE medical SET id = @CON:=@CON+1;



## 위에서 전처리된 medical테이블을 address.csv로 export하여 위도,경도 변환
**토큰은 주기적으로 발급받아줘야함.토큰이 만료되면 실행x**
[사용한 주소변환 API링크](https://sgis.kostat.go.kr/developer/html/newOpenApi/api/dataApi/ready.html)

In [ ]:
import requests as re
import pandas as pd
from pandas import Series, DataFrame,read_csv

csv=pd.read_csv('address.csv',encoding='cp949')

lot= csv['lot']
lat=csv['lat']

tx=[]
ty=[]
for i in range(len(lot)):

    url=f"https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json?src=5181&dst=4326&posX={lot[i]}&posY={lat[i]}&accessToken=your_kostat_accessToken"
    response = re.get(url)
    json_all=response.json()
    json_lot=json_all.get("result").get("posX")
    json_lat=json_all.get("result").get("posY")
    addy=json_lat+0.00269517 #변환된 주소값에 최적의 위치에 마커를 표시하기위해, 구글과 비교하여 오차값을 더해줌
    addx=json_lot+0.00074581 # .. 
    tx.append(addy)
    ty.append(addx)

print(tx)
print(ty)

#위의 내용을 토대로 csv파일을 생성
lotlat_t = pd.DataFrame({'id':csv['id'],'name':csv['name'],'address':csv['address'],'tel':csv['tel'],'lot':tx,'lat':ty})
lotlat_t.to_csv('lotlat_testdf.csv')


전처리된 병원주소 CSV파일을 MYSQL에 적재 / Django에서 사용하기위해 json파일로 export하여
- medical_final.csv  
- medical_list.json
두가지 파일을 생성.

# Django 기능구현(템플릿은 부트스트랩 탬플릿을 사용)

**html의 디자인부분 코드는길어서, django탬플릿의 주석처리되지않은부분을 참고. 따로 ipynb에 기재하지않음**

주로 Medical.html의 지도기능을 중심으로 기재.

In [ ]:
<!-- Medical.html / 병원리스트 출력 -->

    <!-- ======= Alt Services Section 2 /병원리스트 Section ======= -->
    <section id="alt-services-2" class="alt-services section-bg">
      <div class="container" data-aos="fade-up">
          <div class="section-header">
            <h2>병원 상세위치</h2>
          </div>
            <div class="card-body">
                <div class="table-responsive">
                    <table class="table table-bordered" id="dataTable" width="100%" cellspacing="0">
                        <thead>
                            <tr>
                              <th>동물병원 이름</th>
                              <th>주소</th>
                              <th>전화번호</th>
                            </tr>
                        </thead>
                        {% for hospital in hospital_list %}
                        <tbody>
                          <tr>
                            <td>{{hospital.name}}</td>
                            <td>{{hospital.address}}</td>
                            <td>{{hospital.tel}}</td>
                          </tr>
                        </tbody>
                        {% endfor %}
                    </table>
              <!--페이징처리 시작부분-->   
                <ul class="pagination justify-content-center">
                    <!-- 처음페이지 -->
                    {% if hospital_list.has_previous %}
                    <li class="page-item">
                        <a class="page-link" href="?page=1">처음</a>
                    </li>
                    {% else %}
                    <li class="page-item disabled">
                        <a class="page-link" tabindex="-1" aria-disabled="true" href="#">처음</a>
                    </li>
                    {% endif %}
                    <!-- 페이지리스트 -->
                    {% for page_number in hospital_list.paginator.page_range %}
                    {% if page_number >= hospital_list.number|add:-5 and page_number <= hospital_list.number|add:5 %}
                    {% if page_number == hospital_list.number %}
                    <li class="page-item active" aria-current="page">
                        <a class="page-link" href="?page={{ page_number }}">{{ page_number }}</a>
                    </li>
                    {% else %}
                    <li class="page-item">
                        <a class="page-link" href="?page={{ page_number }}">{{ page_number }}</a>
                    </li>
                    {% endif %}
                    {% endif %}
                    {% endfor %}
                    <!-- 마지막페이지 -->
                    {% if hospital_list.has_next %}
                    <li class="page-item">
                        <a class="page-link" href="?page={{ hospital_list.paginator.num_pages }}">마지막</a>
                    </li>
                    {% else %}
                    <li class="page-item disabled">
                        <a class="page-link" tabindex="-1" aria-disabled="true" href="#">마지막</a>
                    </li>
                    {% endif %}
                </ul>
              <!-- 페이징처리 끝 -->
        </div>

    </section><!-- End Alt Services Section 2 /병원리스트Section종료  -->

In [ ]:
#Views.py / 병원리스트 출력

from django.shortcuts import render, redirect
import templates
from .models import *
from django.core.paginator import Paginator
import json

def medical(request):
    #페이지그레이션기능
    #mysql에 적재되어 models.py로불러온 클래스의이름이 MedicalConv이다.
    hospital_list=MediclaConv.objects.only("name", "address","tel").order_by('id') #필요한컬럼만 추출하여 id로정렬
    page =request.GET.get('page','1')
    paginator =Paginator(hospital_list,10)
    page_obj = paginator.get_page(page)
    return render(request,'Medical.html',{'hospital_list':page_obj,})



#Models.py /병원리스트 출력

#mysql에서 불러온 models.py의 class정보는 아래와같다.
class MediclaConv(models.Model):
    name = models.CharField(max_length=1000)
    address = models.CharField(max_length=1000)
    tel = models.CharField(max_length=1000)
    lot = models.DecimalField(max_digits=10, decimal_places=7)
    lat = models.DecimalField(max_digits=10, decimal_places=8)

    class Meta:
        managed = False
        db_table = 'medicla_conv'


In [ ]:
<!-- Medical.html / 카카오 지도기능 -->
    <!-- =======카카오 MAP세션 시작 ======= -->
    <section id="team" class="team">
        <div class="container" data-aos="fade-up">
          <div class="section-header">
            <h2>병원위치</h2>
            <h4>카카오 맵을 통해 마커로 표시됩니다. 이용시, 위치를 항상 켜두세요</h4>
            <button type="button" class="btn bi-check2-all btn-warning" id="address_choice">주소선택하기</button>&nbsp;&nbsp;
            <button type="button" class="btn btn-primary btn bi-check2-all" id="address_current" onclick="getCurrentPosBtn()">현재위치선택</button>&nbsp;&nbsp;
            <button type="button" class="btn btn-success btn bi bi-check2-all" id="movebtn" onclick="getmovebtn()">위치 이동하기</button>
            <div class="form-group">
            <label for="exampleInputEmail1" class="form-label mt-4">선택된 주소</label>
            <input type="text" class="form-control" id="maelcal_address" value="" aria-describedby="addressHelp" placeholder="주소를선택해주세요">
          </div>
          <!--카카오 MAP이 그려지는부분의 시작-->
          <section id="about" class="about">
            <div class="container" data-aos="fade-up">
              <div class="row position-relative">
                <div id="map" style="width:1000x;height:700px;" class ="row position-relative"></div>
                  <script type="text/javascript" src="//dapi.kakao.com/v2/maps/sdk.js?appkey=	your_kakao_appkey&libraries=services"></script>
                  <script>
                   function locationLoadSuccess(pos){
                    // 현재 위치 받아오기
                    var lat=pos.coords.latitude //내위치 lat
                    var lot=pos.coords.longitude //내위치 lot
                    var currentPos = new kakao.maps.LatLng(lat,lot);
                    
                    //가까우면 스무스하게넘어감
                    map.panTo(currentPos);
                    var imageSrc = "https://t1.daumcdn.net/localimg/localimages/07/mapapidoc/markerStar.png"//병원마커와 다른마커
                    var imageSize = new kakao.maps.Size(24, 35);
                    var markerImage2 = new kakao.maps.MarkerImage(imageSrc,imageSize); //현재위치마커이미지정보
                    var locPosition = new kakao.maps.LatLng(lat, lot); //현재나의위치 마커생성
                    var marker = new kakao.maps.Marker({
                      map: map, 
                      position: locPosition,
                      title:"현재위치" ,
                      image : markerImage2 //현재위치마커이미지정보 지정
                    });
                    
                };
                
                  function locationLoadError(pos){
                    alert('위치를 찾지못하여 기본위치로 돌아갑니다.');
                    var locPosition = new kakao.maps.LatLng(35.208442,126.8710661)
                    displayMarker(locPosition); }; //위치를 찾지못했을때 발생하는용도인데 안쓰일수있으니 삭제하셔도됩니다.
  
                  // 위치 가져오기 버튼 클릭시
                  function getCurrentPosBtn(){
                      navigator.geolocation.getCurrentPosition(locationLoadSuccess,locationLoadError)};
                      
                    var mapContainer = document.getElementById('map'), // 지도를 표시할 div
                        mapOption = {
                            center: new kakao.maps.LatLng(35.208442,126.8710661), // 지도의 중심좌표
                            level: 3 // 지도의 확대 레벨.y/x 카카오좌표계:WGS84 경/위도EPSG:4326 ,csv기존좌표계:중부원점(GRS80)	EPSG:5181
                          };
  
                    //마커를 생성하기위해 좌표데이터를 가져오는부분
                  
                    var map = new kakao.maps.Map(mapContainer, mapOption); // 지도를 생성합니다
                    
                    //json파싱작업
                    var lotlats = JSON.parse("{{ lotlatlist|escapejs }}");
                        console.log(lotlats) //데이터 확인용
                    
                    // 마커를 표시할위치와 이름을 배열에담는다
                    var positions = [];
                        for (var i = 0; i < Object.keys(lotlats).length; i++) {
                          var content = {
                            title: lotlats[i].name,
                            latlng: new kakao.maps.LatLng(lotlats[i].lat, lotlats[i].lot),
                            address: lotlats[i].address,
                            tel: lotlats[i].tel,
                          }
                          positions.push(content);
                      };
                      console.log(positions);
  
                     //마커 이미지. 원하시는걸로바꾸셔도됩니다.
                    var imageSrc = "static/img/marker.png";
                    for (var i = 0; i < positions.length; i++) {
                      // 마커 이미지의 이미지 크기 입니다
                      var imageSize = new kakao.maps.Size(30, 40);
                      // 마커 이미지를 생성합니다
                      var markerImage = new kakao.maps.MarkerImage(imageSrc, imageSize);
                      // 마커를 생성합니다
                      var marker = new kakao.maps.Marker({
                        map: map, // 마커를 표시할 지도
                        position: positions[i].latlng, // 마커를 표시할 위치
                        title: positions[i].title, // 마커의 타이틀, 마커에 마우스를 올리면 타이틀이 표시됩니다
                        image: markerImage, // 마커 이미지
                    });
                    
                  //마커 클릭시 작은 팝업창으로 오버레이를띄우는 작업부분
                    (function(marker, place) {
                      // 마크 클릭 시
                      kakao.maps.event.addListener(marker, 'click', function() {
                        var overlay = new kakao.maps.CustomOverlay({
                          // 오버레이에 띄울 내용
                          content: '<div class="wrap">' +
                                '    <div class="info">' +
                                '        <div class="title">' +
                                place.title +
                                '        </div>' +
                                '        <div class="body">' +
                                '            <div class="desc">' + 
                                '                <div class="ellipsis">' + place.address + '</div>' +
                                '		       <div class="jibun ellipsis">' + place.tel + '</div>' +
                                '            </div>' +
                                '        </div>' +
                                '    </div>' +
                                '</div>',
                          map: map,
                          position: marker.getPosition()
                        });
                        // 아무데나 클릭하게되면 overlay를 끄기
                        kakao.maps.event.addListener(map, 'click', function(mouseEvent) {
                          overlay.setMap(null)
                        })
                        console.log(overlay); //확인
                        overlay.setMap(map); //오버레이띄우기
                      })
                    })(marker, positions[i])
                  };
                  document.getElementById("address_choice").addEventListener("click", function(){ //버튼의 아이디가 addresschoice.즉 발생시키고싶은아이디를 넣음
                          new daum.Postcode({
                              oncomplete: function(data) { //입력값을받아올것임
                                  var roadAddr = data.roadAddress;
                                  document.getElementById("maelcal_address").value = data.address; // 선택한 주소를 주소선택창에넣음
                              }  
                          }).open();
                        }); //종료
                 
                //위치 이동하기버튼을 눌렀을때, medical_address안의주소의 위치로 지도를 이동시킴
                function getmovebtn(){
                  // 주소-좌표 변환 객체를 생성합니다
                  var geocoder = new kakao.maps.services.Geocoder();
  
                  // 주소로 좌표를 검색합니다
                  geocoder.addressSearch(document.getElementById("maelcal_address").value, function(result, status) {
  
                      // 정상적으로 검색이 완료됐으면 
                      if (status === kakao.maps.services.Status.OK) {
  
                          var coords = new kakao.maps.LatLng(result[0].y, result[0].x)
  
                          // // 결과값으로 받은 위치를 마커로 표시합니다
                          var imageSrc = "https://t1.daumcdn.net/localimg/localimages/07/mapapidoc/markerStar.png"//병원마커와 다른마커
                          var imageSize = new kakao.maps.Size(24, 35);
                          var markerImage3 = new kakao.maps.MarkerImage(imageSrc,imageSize); 
                          var marker = new kakao.maps.Marker({
                                map: map,
                                position: coords,
                                image: markerImage3
                            });
                          // 지도의 중심을 결과값으로 받은 위치로 이동시킵니다
                          map.setCenter(coords);
                      }
                })};
                  </script>
                  <!--도로명 주소 검색 팝업창을 띄워주는 스크립트-->
                  <script src="//t1.daumcdn.net/mapjsapi/bundle/postcode/prod/postcode.v2.js"></script> 
  
    </main> <!--End #main-->

In [ ]:
# views.py / 카카오 지도기능

def medical(request):
    #카카오지도마커찍기
    with open('static/js/medical_list.json', encoding='utf-8') as json_file:
        lotlat = json.load(json_file)
        lotlatdict = []
    for lotlats in lotlat:
        if lotlats.get('lot'):
            content = {
                "name": lotlats['name'],
                "lot": str(lotlats['lot']),
                "lat": str(lotlats['lat']),
                "address": str(lotlats['address']),
            }
            if lotlats.get('tel'):
                content['tel'] = str(lotlats['tel'])
            else:
                content['tel'] = ''
            lotlatdict.append(content)
    lotlatlist = json.dumps(lotlatdict, ensure_ascii=False)

    return render(request,'Medical.html',{'lotlatlist':lotlatlist})

In [ ]:
# 전체적인 views.py

def medical(request):
    #페이지그레이션기능
    hospital_list=MediclaConv.objects.only("name", "address","tel").order_by('id') #필요한컬럼만 추출하여 id로정렬
    page =request.GET.get('page','1')
    paginator =Paginator(hospital_list,10)
    page_obj = paginator.get_page(page)

    #카카오지도마커찍기
    with open('static/js/medical_list.json', encoding='utf-8') as json_file:
        lotlat = json.load(json_file)
        lotlatdict = []
    for lotlats in lotlat:
        if lotlats.get('lot'):
            content = {
                "name": lotlats['name'],
                "lot": str(lotlats['lot']),
                "lat": str(lotlats['lat']),
                "address": str(lotlats['address']),
            }
            if lotlats.get('tel'):
                content['tel'] = str(lotlats['tel'])
            else:
                content['tel'] = ''
            lotlatdict.append(content)
    lotlatlist = json.dumps(lotlatdict, ensure_ascii=False)

    return render(request,'Medical.html',{'hospital_list':page_obj,'lotlatlist':lotlatlist})